In [1]:
import warnings
warnings.filterwarnings('ignore')
import os

import tws_data 
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from ib_insync import *
import numpy as np
import pandas as pd
import datetime as dt
import time
pd.options.display.float_format = '{:,.4f}'.format
%matplotlib inline
from scipy.stats import norm as norm
import matplotlib.pyplot as plt
import requests as req
from bs4 import BeautifulSoup as bs
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import math


In [2]:
os.chdir('D:\Options Data\IB Intraday')

def create_dfrow(curr_option):
    df_row = pd.DataFrame({'Symbol': curr_option.contract.localSymbol, 
                           'Type':curr_option.contract.right, 
                           'Bid': curr_option.bid, 
                           'Ask': curr_option.ask, 
                           'Volume': curr_option.volume, 
                           'Strike': curr_option.contract.strike,
                           'Expiry': dt.datetime.strptime(curr_option.contract.lastTradeDateOrContractMonth, '%Y%m%d'),
                           'bidIV': curr_option.bidGreeks.impliedVol, 
                           'bidDelta': curr_option.bidGreeks.delta,
                           'bidGamma': curr_option.bidGreeks.gamma, 
                           'bidVega': curr_option.bidGreeks.vega,
                           'bidTheta': curr_option.bidGreeks.theta, 
                           'askIV': curr_option.askGreeks.impliedVol, 
                           'askDelta': curr_option.askGreeks.delta,
                           'askGamma': curr_option.askGreeks.gamma, 
                           'askVega': curr_option.askGreeks.vega,
                           'askTheta': curr_option.askGreeks.theta,
                           'modelIV': curr_option.modelGreeks.impliedVol, 
                           'modelDelta': curr_option.modelGreeks.delta,
                           'modelGamma': curr_option.modelGreeks.gamma, 
                           'modelVega': curr_option.modelGreeks.vega,
                           'modelTheta': curr_option.modelGreeks.theta}, index = [0])
    return df_row

def get_movers():

    indexarb_url = 'http://www.indexarb.com/stockVsIndexesMiddleColumn.html#djIndexAnalysis'
    indexarb_soup = bs(req.get(indexarb_url).text,'lxml')
    indexarb_tables = indexarb_soup.select('table[align="LEFT"]')

    movers = []

    for i, table_html in enumerate(indexarb_tables):
        curr_table = pd.read_html(str(table_html))[2]
        curr_table.columns = curr_table.loc[1,:].tolist()

        if i == 0:
            curr_table = curr_table.iloc[2:,:4].set_index('Index').apply(pd.to_numeric)
        else:
            curr_table = curr_table.iloc[2:,[1,3,4,5,6]].dropna().set_index('Sym').apply(pd.to_numeric)

        curr_table.columns = [x.split(' ')[0] for x in curr_table.columns.tolist()]
        movers.append(curr_table)

    index = movers[0]
    movers = pd.concat(movers[1:], axis = 0)
    movers = movers.sort_values('OneMonth')
    
    return index, movers

def time_remaining(weekly_options, curr_spx):
    typ = weekly_options.reset_index().loc[0,'Type']

    try:
        interest_rate = bs(req.get('https://www.marketwatch.com/investing/bond/tmubmusd01m?countrycode=bx','lxml').text, 'lxml')
        interest_rate = float(interest_rate.select_one('h3[class*="intraday__price"]').select_one('bg-quote').text)/100
    except:
        interest_rate = 0.02422

    weekly_options['a'] = interest_rate - (weekly_options['modelIV']**2)/2

    if typ == 'P':
        weekly_options['b'] = weekly_options['modelIV']*norm.ppf(-weekly_options['modelDelta'])
    if typ == 'C':
        weekly_options['b'] = -weekly_options['modelIV']*norm.ppf(weekly_options['modelDelta'])
    weekly_options['c'] = np.log(curr_spx/weekly_options['Strike'])

    weekly_options_otm = weekly_options[abs(weekly_options.modelDelta) <= 0.5]
    weekly_options_itm = weekly_options[abs(weekly_options.modelDelta) > 0.5]

    if typ == 'P':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b - 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b + 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    if typ == 'C':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b + 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b - 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    weekly_times = pd.concat([weekly_options_otm, 
                              weekly_options_itm], axis = 0)[['Expiry','time_remaining']].dropna().groupby('Expiry').mean()

    del weekly_options['a'], weekly_options['b'], weekly_options['c'], weekly_options_itm, weekly_options_otm

    return weekly_options.merge(weekly_times.reset_index(), on = 'Expiry')


def get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False):
    if contractType == 'Stock':
        security = Stock(ticker,exchange)
    else:
        security = Index(ticker, exchange)

    ib.qualifyContracts(security)

    chains = ib.reqSecDefOptParams(security.symbol, '', security.secType, security.conId)

    chains_df = util.df(chains)
    chain = next(c for c in chains if c.tradingClass == ticker and c.exchange == 'SMART')

    all_expirations = sorted(exp for exp in chain.expirations)
    expirations_dates = [dt.datetime.strptime(x, '%Y%m%d') for x in all_expirations]

    expirations = list(filter(lambda x: x[0] >= earliest_date and x[0] <= latest_date, list(zip(expirations_dates,all_expirations))))
    dte_expirations = [(expir[0] - dt.datetime.today()).days for expir in expirations]
    expirations = [expir[1] for expir in expirations]

    ib.reqHeadTimeStamp(security, whatToShow='TRADES', useRTH=True)

    end_date = '' #'20100506 13:00:00 PST'
    duration = '1 D'

    sec_bars = ib.reqHistoricalData(
            security,
            endDateTime=end_date,
            durationStr=duration,
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1)


    sec_df = util.df(sec_bars).set_index('date')
    sec_df.index = pd.to_datetime(sec_df.index)

    curr_sec_price = sec_df.reset_index().loc[len(sec_df) - 1, 'close']
    bounds_lst = [curr_sec_price*(stdevs*stdevs/(100*np.sqrt(365.25/dte))) for dte in dte_expirations]


    contracts_list = []

    for bounds, expiration in zip(bounds_lst, expirations):
#         strikes = [strike for strike in chain.strikes
#                    if strike % 5 == 0
#                    and curr_sec_price - bounds < strike < curr_sec_price + bounds]
        strikes = [strike for strike in chain.strikes
                   if curr_sec_price - bounds < strike < curr_sec_price + bounds]

        rights = ['P', 'C']

        curr_contracts = [Option(ticker, expiration, strike, right, 'SMART')
                          for right in rights
                          for strike in strikes]
        contracts_list += [curr_contracts]

    puts_list = []
    calls_list = []

    for contracts in contracts_list:
        ib.qualifyContracts(*contracts)
        sec_options = ib.reqTickers(*contracts)

        puts_rows = []
        calls_rows = []

        for curr_option in sec_options:
            try:
                curr_row = create_dfrow(curr_option)
                if curr_option.contract.right == 'P':
                    puts_rows.append(curr_row)
                elif curr_option.contract.right == 'C':
                    calls_rows.append(curr_row)
            except:
                continue

        puts = pd.concat(puts_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        calls = pd.concat(calls_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        puts = time_remaining(puts, curr_sec_price)
        calls = time_remaining(calls, curr_sec_price)
        puts_list.append(puts)
        calls_list.append(calls)
        time.sleep(5)

    puts_df = pd.concat(puts_list,axis = 0).reset_index(drop = True)
    calls_df = pd.concat(calls_list,axis = 0).reset_index(drop = True)

    if save_csv:
        curr_time = dt.datetime.now().strftime('%Y%m%d-%H-%M')
        puts_df.to_csv('ib_puts' + curr_time + '.csv')
        calls_df.to_csv('ib_calls' + curr_time + '.csv')
        
    return puts_df, calls_df, curr_sec_price


def get_spreads(weekly_options, curr_spx, expiry_index = 0, rowshift = 1, commissions = 0.0266):

    expirations_dates = weekly_options.Expiry.drop_duplicates().tolist()

    weekly_options = weekly_options[weekly_options.Expiry == expirations_dates[expiry_index]]

    typ = weekly_options.reset_index().loc[0,'Type']

    if typ == 'P':
        weekly_options = weekly_options.sort_values('Strike', ascending = False).reset_index(drop = True)
    if typ == 'C':
        weekly_options = weekly_options.sort_values('Strike', ascending = True).reset_index(drop = True)

    time_remaining = weekly_options.reset_index(drop = True).loc[0,'time_remaining']

    weekly_options['Mid'] = (weekly_options.Ask + weekly_options.Bid)/2
    shorts = weekly_options[['Mid', 'Strike', 'bidDelta', 'bidGamma', 'bidTheta', 'bidVega']]
    shorts.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']
    longs = weekly_options[['Mid','Strike', 'askDelta', 'askGamma','askTheta', 'askVega']]
    longs.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']

    spreads = shorts - longs.shift(-rowshift)
    spreads['Short_Strike'] = shorts.Strike
    spreads['Short_Delta'] = shorts.Delta
    spreads['Long_Strike'] = longs.Strike.shift(-rowshift)
    spreads = spreads[['Short_Strike', 'Long_Strike', 'Short_Delta', 'Price',
                       'Strike', 'Delta', 'Gamma', 'Theta', 'Vega']]

    spreads['Credit'] = spreads.Price - commissions

    if typ == 'P':
        spreads['MaxLoss'] = (-spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] - spreads.Credit
    if typ == 'C':
        spreads['MaxLoss'] = (spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] + spreads.Credit

    del spreads['Price'], spreads['Strike']

    spreads = spreads[spreads.Credit > 0].dropna().reset_index(drop = True)
    spreads['Expiry'] = expirations_dates[expiry_index]

    return spreads

def art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias = 'upside',
               maxshift = 6, commissions = 0.0266):
    
    hedge = []
    for rowshift in range(1,maxshift):

        if bias == 'upside':
            spreads = get_spreads(curr_puts, curr_price, expiry_index, rowshift, commissions = 0.0266)
        else:
            spreads = get_spreads(curr_calls, curr_price, expiry_index, rowshift, commissions = 0.0266)
        spreads = spreads[(spreads.Credit > 0.5)]
        hedge.append(spreads)

    hedge = pd.concat(hedge, axis = 0).reset_index(drop = True)

    if bias == 'upside':
        expirations_dates = curr_calls.Expiry.drop_duplicates().tolist()

        long_hedge = curr_calls[(curr_calls.Expiry == expirations_dates[expiry_index])][['Ask', 'Bid', 'Expiry', 'Strike',
                                                                             'modelDelta', 'modelGamma',
                                                                             'modelTheta', 'modelVega', 'time_remaining']].reset_index(drop = True)
    else:
        expirations_dates = curr_puts.Expiry.drop_duplicates().tolist()

        long_hedge = curr_puts[(curr_puts.Expiry == expirations_dates[expiry_index])][['Ask', 'Bid', 'Expiry', 'Strike',
                                                                             'modelDelta', 'modelGamma',
                                                                             'modelTheta', 'modelVega', 'time_remaining']].reset_index(drop = True)

    long_hedge['Mid'] = (long_hedge['Ask'] + long_hedge['Bid'])/2
    long_hedge.columns = [x.replace('model','') for x in long_hedge.columns.tolist()]

    hedges_lst = []

    for idx, row in hedge.iterrows():

        curr_hedge = long_hedge[long_hedge.Mid < row.Credit][['Mid','Expiry', 'Strike',
                                                              'Delta', 'Gamma',
                                                              'Theta', 'Vega', 'time_remaining']]
        curr_hedge['Gamma'] = curr_hedge['Gamma'] - row.Gamma
        curr_hedge['Delta'] = curr_hedge['Delta'] - row.Delta
        curr_hedge['Theta'] = curr_hedge['Theta'] - row.Theta
        curr_hedge['Vega'] = curr_hedge['Vega'] - row.Vega
        if bias == 'upside':
            curr_hedge = curr_hedge[(curr_hedge.Delta > 0) &
                                    (curr_hedge.Gamma > 0) &
                                    (curr_hedge.Theta > 0) &
                                    (curr_hedge.Vega > 0)]
        else:
            curr_hedge = curr_hedge[(curr_hedge.Delta < 0) &
                                (curr_hedge.Gamma > 0) &
                                (curr_hedge.Theta > 0) &
                                (curr_hedge.Vega > 0)]
        curr_hedge['Credit'] = -curr_hedge.Mid + row.Credit - 0.0133

        if bias == 'upside':
            curr_hedge['MaxLoss'] = (row.Long_Strike - row.Short_Strike + curr_hedge.Credit)*100
            curr_hedge['BreakEven'] = row.Short_Strike - curr_hedge['Credit']
        else:
            curr_hedge['MaxLoss'] = (row.Short_Strike - row.Long_Strike + curr_hedge.Credit)*100
            curr_hedge['BreakEven'] = row.Short_Strike + curr_hedge['Credit']
        curr_hedge['Maturity'] = curr_hedge['Expiry']
        curr_hedge['Mid'] = row.Short_Strike
        curr_hedge['Expiry'] = row.Long_Strike

        if bias == 'upside':
            curr_hedge.columns = ['Short_Put_Strike','Long_Put_Strike','Long_Call_Strike'] + curr_hedge.columns.tolist()[3:]
        else:
            curr_hedge.columns = ['Short_Call_Strike','Long_Call_Strike','Long_Put_Strike'] + curr_hedge.columns.tolist()[3:]

        hedges_lst.append(curr_hedge)

    hedges_df = pd.concat(hedges_lst, axis = 0)
    
    return hedges_df

In [3]:
index, movers = get_movers()

In [4]:
losers = movers.head()
winners = movers.tail().sort_values('OneMonth', ascending = False)

In [3]:
client_id = 12

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=client_id)

<IB connected to 127.0.0.1:7496 clientId=12>

In [4]:
# exchange = 'NYSE'
# contractType = 'Stock'
# earliest_date = dt.datetime(2019,4,6)
# latest_date = dt.datetime(2019,6,30)
# curr_iv = 40
# stdevs = 9

# stock_prices = {}
# stock_puts = {}
# stock_calls = {}

# for ticker in losers.index:

#     curr_puts, curr_calls, curr_price = get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False)
    
#     stock_prices[ticker] = curr_price
#     stock_puts[ticker] = curr_puts
#     stock_calls[ticker] = curr_calls
#     break

In [4]:
exchange = 'NYSE'
contractType = 'Stock'
earliest_date = dt.datetime(2019,11,3)
latest_date = dt.datetime(2019,11,18)

ticker = 'BYND'
curr_iv = 150
stdevs = 30
curr_puts, curr_calls, curr_price = get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False)

Started to throttle requests
Error 200, reqId 9: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=22.5, right='P', exchange='SMART')
Error 200, reqId 10: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=25.0, right='P', exchange='SMART')
Error 200, reqId 11: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=30.0, right='P', exchange='SMART')
Error 200, reqId 12: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=35.0, right='P', exchange='SMART')
Error 200, reqId 14: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=42.5, right='P', exchange='SMART')
Error 200, reqId 

Error 200, reqId 63: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=93.0, right='P', exchange='SMART')
Error 200, reqId 64: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=94.0, right='P', exchange='SMART')
Error 200, reqId 66: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=96.0, right='P', exchange='SMART')
Error 200, reqId 67: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=97.0, right='P', exchange='SMART')
Error 200, reqId 68: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=98.0, right='P', exchange='SMART')
Error 200, reqId 69: No security definition h

Error 200, reqId 173: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=61.0, right='C', exchange='SMART')
Error 200, reqId 174: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=62.0, right='C', exchange='SMART')
Error 200, reqId 175: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=63.0, right='C', exchange='SMART')
Error 200, reqId 176: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=64.0, right='C', exchange='SMART')
Error 200, reqId 177: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=62.5, right='C', exchange='SMART')
Error 200, reqId 178: No security defini

Error 200, reqId 225: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=106.0, right='C', exchange='SMART')
Error 200, reqId 226: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=107.0, right='C', exchange='SMART')
Error 200, reqId 227: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=108.0, right='C', exchange='SMART')
Error 200, reqId 228: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=109.0, right='C', exchange='SMART')
Error 200, reqId 230: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=111.0, right='C', exchange='SMART')
Error 200, reqId 278: No security d

Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=81.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=82.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=82.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=83.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=84.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=86.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=87.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191108', strike=87.5, right='P', exchange='SMART')
Unknown contract

...

Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=92.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=93.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=94.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=96.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=97.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=98.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=124.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=126.0, right='P', exchange='SMART')
Unknown contra

Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=91.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=92.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=93.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=94.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=96.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=97.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=98.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=124.0, right='C', exchange='SMART')
Unknown contrac

Error 200, reqId 947: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=76.0, right='P', exchange='SMART')
Error 200, reqId 948: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=77.0, right='P', exchange='SMART')
Error 200, reqId 950: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=78.0, right='P', exchange='SMART')
Error 200, reqId 951: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=79.0, right='P', exchange='SMART')
Error 200, reqId 953: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=81.0, right='P', exchange='SMART')
Error 200, reqId 954: No security defini

Error 200, reqId 1029: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=167.5, right='P', exchange='SMART')
Error 200, reqId 1031: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=172.5, right='P', exchange='SMART')
Error 200, reqId 1033: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=177.5, right='P', exchange='SMART')
Error 200, reqId 1035: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=182.5, right='P', exchange='SMART')
Error 200, reqId 1037: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=187.5, right='P', exchange='SMART')
Error 200, reqId 1039: No secu

Error 200, reqId 1126: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=98.0, right='C', exchange='SMART')
Error 200, reqId 1150: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=124.0, right='C', exchange='SMART')
Error 200, reqId 1152: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=126.0, right='C', exchange='SMART')
Error 200, reqId 1153: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=127.0, right='C', exchange='SMART')
Error 200, reqId 1154: No security definition has been found for the request, contract: Option(symbol='BYND', lastTradeDateOrContractMonth='20191115', strike=128.0, right='C', exchange='SMART')
Error 200, reqId 1155: No secur

In [5]:
maxshift = 50

arteHedges_upside = []
for expiry_index in range(len(curr_calls.Expiry.drop_duplicates())):
    bias = 'upside'
    possible_pos = art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias,
                              maxshift = maxshift, commissions = 0.0266)
    arteHedges_upside.append(possible_pos)

arteHedges_upside = pd.concat(arteHedges_upside,axis = 0)

In [6]:
arteHedges_downside = []
for expiry_index in range(len(curr_calls.Expiry.drop_duplicates())):
    bias = 'downside'
    possible_pos = art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias,
                              maxshift = maxshift, commissions = 0.0266)
    arteHedges_downside.append(possible_pos)
arteHedges_downside = pd.concat(arteHedges_downside,axis = 0)

arteHedges_upside['ReturnOnRisk'] = abs(arteHedges_upside.Credit/arteHedges_upside.MaxLoss)
arteHedges_downside['ReturnOnRisk'] = abs(arteHedges_downside.Credit/arteHedges_downside.MaxLoss)

In [ ]:
potential_ups = arteHedges_upside[(arteHedges_upside.Short_Put_Strike < curr_price) &
                                  (arteHedges_upside.Short_Put_Strike > 0.001)].sort_values(['Maturity', 
                                                                                             'ReturnOnRisk'], ascending = False)
try:
    potential_ups['downmove_to_breach'] = potential_ups['Short_Put_Strike']/curr_price - 1
    potential_ups['upmove_to_win'] = potential_ups['Long_Call_Strike']/curr_price - 1
    potential_ups = potential_ups.sort_values('MaxLoss').reset_index(drop = True)
    potential_ups['MaxLossScore'] = potential_ups.index
    
    potential_ups = potential_ups.sort_values('downmove_to_breach', ascending = False).reset_index(drop = True)
    potential_ups['DownMoveScore'] = potential_ups.index
    
    potential_ups = potential_ups.sort_values('upmove_to_win', ascending = False).reset_index(drop = True)
    potential_ups['UpMoveScore'] = potential_ups.index
    
    potential_ups['Score'] = potential_ups.MaxLossScore + potential_ups.DownMoveScore + potential_ups.UpMoveScore
    potential_ups = potential_ups.sort_values('Score', ascending = False)
except:
    None
potential_ups

In [7]:
potential_downs = arteHedges_downside[(arteHedges_downside.Short_Call_Strike > curr_price)].sort_values(['Maturity', 
                                                                                                         'Long_Put_Strike'], ascending = False)
try:
    potential_downs['upmove_to_breach'] = potential_downs['Short_Call_Strike']/curr_price - 1
    potential_downs['downmove_to_win'] = potential_downs['Long_Put_Strike']/curr_price - 1
    potential_downs = potential_downs.sort_values('MaxLoss').reset_index(drop = True)
    potential_downs['MaxLossScore'] = potential_downs.index
    
    potential_downs = potential_downs.sort_values('upmove_to_breach').reset_index(drop = True)
    potential_downs['UpMoveScore'] = potential_downs.index
    
    potential_downs = potential_downs.sort_values('downmove_to_win').reset_index(drop = True)
    potential_downs['DownMoveScore'] = potential_downs.index
    
    potential_downs['Score'] = potential_downs.MaxLossScore + potential_downs.UpMoveScore + potential_downs.DownMoveScore
    potential_downs = potential_downs.sort_values('Score', ascending = False)
except:
    None
potential_downs

,Short_Call_Strike,Long_Call_Strike,Long_Put_Strike,Delta,Gamma,Theta,Vega,time_remaining,Credit,MaxLoss,BreakEven,Maturity,ReturnOnRisk,upmove_to_breach,downmove_to_win,MaxLossScore,UpMoveScore,DownMoveScore,Score
18,106.0000,118.0000,42.5000,-0.1811,0.0000,0.0036,0.0006,0.0505,3.8201,-817.9900,109.8201,2019-11-15,0.0047,0.0267,-0.5883,19,25,18,62
4,107.0000,114.0000,35.0000,-0.1045,0.0001,0.0004,0.0018,0.0505,2.3801,-461.9900,109.3801,2019-11-15,0.0052,0.0364,-0.6610,28,28,4,60
19,106.0000,119.0000,45.0000,-0.1969,0.0000,0.0028,0.0013,0.0505,3.9501,-904.9900,109.9501,2019-11-15,0.0044,0.0267,-0.5641,13,27,19,59
25,105.0000,119.0000,47.5000,-0.2146,0.0003,0.0001,0.0057,0.0505,4.3301,-966.9900,109.3301,2019-11-15,0.0045,0.0170,-0.5399,7,22,25,54
22,105.0000,118.0000,45.0000,-0.1984,0.0003,0.0008,0.0013,0.0505,4.1501,-884.9900,109.1501,2019-11-15,0.0047,0.0170,-0.5641,14,18,22,54
1,106.0000,114.0000,35.0000,-0.1192,0.0001,0.0008,0.0023,0.0505,2.8051,-519.4900,108.8051,2019-11-15,0.0054,0.0267,-0.6610,27,26,1,54
27,105.0000,125.0000,57.5000,-0.3082,0.0001,0.0028,0.0035,0.0505,5.3801,"-1,461.9900",110.3801,2019-11-15,0.0037,0.0170,-0.4430,2,23,27,52
23,105.0000,119.0000,45.0000,-0.2119,0.0002,0.0043,0.0018,0.0505,4.4001,-959.9900,109.4001,2019-11-15,0.0046,0.0170,-0.5641,8,21,23,52
14,105.0000,117.0000,42.5000,-0.1814,0.0003,0.0002,0.0007,0.0505,3.9951,-800.4900,108.9951,2019-11-15,0.0050,0.0170,-0.5883,20,17,14,51
7,105.0000,118.0000,40.0000,-0.1942,0.0001,0.0084,0.0009,0.0505,4.3151,-868.4900,109.3151,2019-11-15,0.0050,0.0170,-0.6126,16,24,7,47


In [ ]:
potential_ups[(potential_ups.MaxLoss > - 1000) &
              (potential_ups.Long_Call_Strike < 150)].sort_values('Short_Put_Strike')

In [65]:
ib.disconnect()